In [ ]:
"""
Zadanie 2.

Przygotuj chmurę słów (lub wykres kolumnowy dla 50 najczęściej używanych słów) w oparciu o *bag-of-words*. Dla pozyskanych przez Ciebie dokumentów (w języku angielskim). Korpus ma odzwierciedlać pewną dychotomię - konflikt, dyskusję dwóch stron pewnego zjawiska np: zmian klimatycznych, lockdownu, ulubionego sportu (rugby vs. piłka nożna). Mogą to być: teksty tweetów na wybrane tematy (min 1000 tweetów dla każdej strony); dwóch grup artykułów (po 3-5 dla każdej strony, każdy ponad 2000 wyrazów); wypowiedzi dwóch ekspertów czy polityków (po 3-5 dla każdej strony, każdy ponad 2000 wyrazów).

Analizę wykonaj w dwóch wersjach:

1. dwa teksty traktowane są osobno - jako dwa osobne korpusy, przygotuj dwie wizualizacje dla każdej strony osobno.
2. dwa teksty traktowane są jako jeden i wykonaj wizualizację prezentującą:

    - termów charakterystycznych dla każdej ze stron (termy pojawiające się w wypowiedziach jednej strony ale nie pojawiające się w wypowiedziach drugiej strony),
    - termów wspólnych dla dwóch stron (termy pojawiające się w wypowiedziach jednej i drugiej strony jednocześnie).
"""

In [5]:
import twint
import nest_asyncio
import twint
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [6]:
nest_asyncio.apply()

In [7]:
c = twint.Config()
c.Search = '#verstappen'
c.Limit = 100
twint.run.Search(c)

1640086489253007365 2023-03-26 22:21:05 +0200 <Formula1newsUK> Lewis Hamilton admits ‘shock’ as he issues warning to Max #Verstappen  https://t.co/XX5s3xWUFb
1640067253474414598 2023-03-26 21:04:38 +0200 <Publisport_MX> Max Verstappen sufría duros castigos de niño cuando no ganaba carreras #verstappen #F1    https://t.co/cGBQ9AnKUC
1640062079921975297 2023-03-26 20:44:05 +0200 <amr23fa14> would Compare Daniel Ricciardo to UFC Tyron Woodley, both had all the talent in the world but success got into their head !   in sports, no space for Talent without discipline …  #f1 #verstappen #alonso #AusGP
1640032489111535618 2023-03-26 18:46:30 +0200 <ACM01985> Ahora resulta que tiene problemas psicologicos #verstappen #f1 #checo #RedBull
1640030553461096448 2023-03-26 18:38:48 +0200 <Formula1newsUK> #Ferrari boss claims Sergio Perez could ‘easily’ become 100 points ahead of Max #Verstappen  https://t.co/SjC0LVcqm3
1640029614591311872 2023-03-26 18:35:05 +0200 <DonBalon> 🏆 La historia que avala e

In [8]:
# Definicja funkcji pobierającej dane za pomocą biblioteki twint
def get_data(query, limit):
    c = twint.Config()
    c.Search = query
    c.Lang = 'en'
    c.Limit = limit
    c.Pandas = True
    twint.run.Search(c)
    tweets = twint.storage.panda.Tweets_df
    return tweets

# Definicja funkcji generującej chmurę słów
def generate_wordcloud(data):
    wordcloud = WordCloud(background_color="white", max_words=50, contour_width=3, contour_color='steelblue')
    wordcloud.generate(data)
    plt.figure(figsize = (8, 8), facecolor = 'k', edgecolor = 'k')
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.show()

In [ ]:
# Pobieranie tweetów na temat zmian klimatycznych
query1 = 'climate change'
limit1 = 1000
tweets1 = get_data(query1, limit1)

# Generowanie chmury słów dla tweetów na temat zmian klimatycznych
text1 = ' '.join(tweet for tweet in tweets1.tweet)
generate_wordcloud(text1)

# Pobieranie tweetów na temat lockdownu
query2 = 'lockdown'
limit2 = 1000
tweets2 = get_data(query2, limit2)

# Generowanie chmury słów dla tweetów na temat lockdownu
text2 = ' '.join(tweet for tweet in tweets2.tweet)
generate_wordcloud(text2)

In [16]:
def get_data(query, limit):
    c = twint.Config()
    c.Search = query
    c.Lang = 'en'
    c.Limit = limit
    c.Pandas = True
    c.Store_articles = True
    twint.run.Search(c)
    articles = twint.storage.panda.Tweets_df
    return articles

In [17]:
# Pobieranie artykułów na temat rugby
query1 = 'rugby'
limit1 = 5
articles1 = get_data(query1, limit1)

# Pobieranie artykułów na temat piłki nożnej
query2 = 'soccer'
limit2 = 5
articles2 = get_data(query2, limit2)

# Łączenie tekstów z dwóch źródeł
text1 = ' '.join(article for article in articles1.article)
text2 = ' '.join(article for article in articles2.article)
text = text1 + ' ' + text2

# Generowanie chmury słów dla terminów charakterystycznych dla każdej ze stron
wordcloud = WordCloud(background_color="white", max_words=50, contour_width=3, contour_color='steelblue')
wordcloud.generate(text)
plt.figure(figsize = (8, 8), facecolor = 'k', edgecolor = 'k')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

1640093864819150852 2023-03-26 22:50:23 +0200 <RedruthRugby> Next home game 🔴  https://t.co/kQ2bDGgZDd
1640093819478704128 2023-03-26 22:50:12 +0200 <sethGBR> This ref doesn’t blow the whistle until the shooter flails upon landing.  Calling a touch foul to decide a rugby match is disgusting.  https://t.co/xMXNdaoEV9
1640093813535571968 2023-03-26 22:50:11 +0200 <PuigJnr> @ScottPatten76 @telegraph_sport France Rugby Union to France Rugby League pot kettle. Union can only blame themselves way back in 1907, should of paid the players then League wouldn't of formed
1640093792760995840 2023-03-26 22:50:06 +0200 <DiscipleElland> @BrynLaw There's actually lots on it Bryn. They show La Liga plus all the Scottish Cup &amp; League Cup games. (Big viewing figures Celtic &amp; Rangers fanbases). They also show PDC Darts &amp; Rugby etc...
1640093783025917952 2023-03-26 22:50:03 +0200 <Onemola> The ‘Haka’ performed by the All Blacks — New Zealand’s national rugby team.    https://t.co/CxZjQNieua
16

AttributeError: 'DataFrame' object has no attribute 'article'

In [15]:
# Podział tekstu na dwa podkorpusy
text1 = ' '.join(article for article in articles1.article)
text2 = ' '.join(article for article in articles2.article)

# Generowanie chmury słów dla terminów charakterystycznych dla rugby
wordcloud1 = WordCloud(background_color="white", max_words=50, contour_width=3, contour_color='steelblue')
wordcloud1.generate(text1)
plt.figure(figsize = (8, 8), facecolor = 'k', edgecolor = 'k')
plt.imshow(wordcloud1)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

# Generowanie chmury słów dla terminów charakterystycznych dla piłki nożnej
wordcloud2 = WordCloud(background_color="white", max_words=50, contour_width=3, contour_color='steelblue')
wordcloud2.generate(text2)

AttributeError: 'DataFrame' object has no attribute 'article'